In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
train_data=pd.read_csv("/kaggle/input/prediction-bash/train.csv")
test_data=pd.read_csv("/kaggle/input/prediction-bash/test.csv")

In [ ]:
from sklearn.impute import SimpleImputer
numeric_cols = ['cols_2', 'cols_8', 'cols_9', 'cols_10', 'cols_11', 'cols_12']
imputer_numeric = SimpleImputer(strategy='mean')
train_data[numeric_cols] = imputer_numeric.fit_transform(train_data[numeric_cols])

categorical_cols = ['cols_1', 'cols_4', 'cols_5', 'cols_6', 'cols_7', 'cols_13', 'cols_14']
imputer_categorical = SimpleImputer(strategy='most_frequent')
train_data[categorical_cols] = imputer_categorical.fit_transform(train_data[categorical_cols])


In [ ]:
train_data["cols_1"].unique()

In [ ]:
for i in train_data:
    print(train_data[i].unique())

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
train_data["cols_1"]=le.fit_transform(train_data["cols_1"])
train_data["cols_3"]=le.fit_transform(train_data["cols_3"])
train_data["cols_4"]=le.fit_transform(train_data["cols_4"])
train_data["cols_5"]=le.fit_transform(train_data["cols_5"])
train_data["cols_6"]=le.fit_transform(train_data["cols_6"])
train_data["cols_7"]=le.fit_transform(train_data["cols_7"])
train_data["cols_13"]=le.fit_transform(train_data["cols_13"])
train_data["cols_14"]=le.fit_transform(train_data["cols_14"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler
numeric_cols = ['cols_2', 'cols_3', 'cols_4', 'cols_5', 'cols_6', 'cols_7', 'cols_8', 'cols_9', 'cols_10', 'cols_11', 'cols_12', 'cols_13', 'cols_14']
scaler = MinMaxScaler()
train_data[numeric_cols] = scaler.fit_transform(train_data[numeric_cols])

In [ ]:
df=train_data

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error

X = df.drop(columns=['id', 'True_value'])
y = df['True_value']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_scaled = X_train
X_test_scaled = X_test


# Hyperparameter Tuning
param_grid = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}
grid_search = GridSearchCV(RandomForestRegressor(random_state=42), param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search.fit(X_train_scaled, y_train)
best_regressor = grid_search.best_estimator_

# Fitting the model
best_regressor.fit(X_train_scaled, y_train)

# Predictions
y_pred = best_regressor.predict(X_test_scaled)

# Mean Absolute Error
mae = mean_absolute_error(y_test, y_pred)
print("Mean Absolute Error:", mae)

In [ ]:
import matplotlib.pyplot as plt

# Calculate the minimum and maximum values of true values and predictions
min_value = min(y_test.min(), y_pred.min())
max_value = max(y_test.max(), y_pred.max())

plt.scatter(y_test, y_pred)
plt.xlabel("True Values")
plt.ylabel("Predictions")
plt.title("True Values vs. Predictions")

# Set the same limits for both axes
plt.xlim(min_value, max_value)
plt.ylim(min_value, max_value)

# Plot the diagonal line for reference
plt.plot([min_value, max_value], [min_value, max_value], color='red', linestyle='--')

plt.show()


In [ ]:
test_data

In [ ]:
from sklearn.impute import SimpleImputer
numeric_cols = ['cols_2', 'cols_8', 'cols_9', 'cols_10', 'cols_11', 'cols_12']
imputer_numeric = SimpleImputer(strategy='mean')
test_data[numeric_cols] = imputer_numeric.fit_transform(test_data[numeric_cols])
categorical_cols = ['cols_1', 'cols_4', 'cols_5', 'cols_6', 'cols_7', 'cols_13', 'cols_14']
imputer_categorical = SimpleImputer(strategy='most_frequent')
test_data[categorical_cols] = imputer_categorical.fit_transform(test_data[categorical_cols])

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
test_data["cols_1"]=le.fit_transform(test_data["cols_1"])
test_data["cols_3"]=le.fit_transform(test_data["cols_3"])
test_data["cols_4"]=le.fit_transform(test_data["cols_4"])
test_data["cols_5"]=le.fit_transform(test_data["cols_5"])
test_data["cols_6"]=le.fit_transform(test_data["cols_6"])
test_data["cols_7"]=le.fit_transform(test_data["cols_7"])
test_data["cols_13"]=le.fit_transform(test_data["cols_13"])
test_data["cols_14"]=le.fit_transform(test_data["cols_14"])

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Select numeric columns for standard scaling
numeric_cols = ['cols_2', 'cols_3', 'cols_4', 'cols_5', 'cols_6', 'cols_7', 'cols_8', 'cols_9', 'cols_10', 'cols_11', 'cols_12', 'cols_13', 'cols_14']

# Initialize the StandardScaler
scaler = MinMaxScaler()

# Fit and transform the data
test_data[numeric_cols] = scaler.fit_transform(test_data[numeric_cols])


In [ ]:
x=[]
predictions = best_regressor.predict(test_data.drop(columns=["id"]))
for i in predictions:
    x.append(i)
x

In [ ]:
sample=pd.DataFrame()

In [ ]:
sample["id"]=test_data["id"]
sample['True_value']=x

In [62]:
submission=sample


In [64]:
sample

,id,True_value
0,Id_6252,87062.033321
1,Id_4684,150589.689347
2,Id_1731,50540.439605
3,Id_4742,27300.919664
4,Id_4521,153779.974173
...,...,...
3995,Id_4209,108299.555628
3996,Id_681,38953.057066
3997,Id_434,55615.994289
3998,Id_9309,49817.996026


In [67]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(1, activation='relu'))

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_4 (Dense)                 │ (None, 64)             │           960 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,585 (14.00 KB)

 Trainable params: 3,585 (14.00 KB)

 Non-trainable params: 0 (0.00 B)

In [68]:
model.fit(x=X_train,y=y_train,validation_split=0.3,batch_size=10,epochs=100,shuffle=True,verbose=2)

Epoch 1/100
336/336 - 2s - 6ms/step - loss: 97674.5547 - mae: 97674.6250 - val_loss: 86559.9531 - val_mae: 86559.9688
Epoch 2/100
336/336 - 1s - 3ms/step - loss: 60303.8281 - mae: 60303.7852 - val_loss: 50638.8906 - val_mae: 50638.8984
Epoch 3/100
336/336 - 1s - 2ms/step - loss: 49079.2031 - mae: 49079.1875 - val_loss: 48093.4531 - val_mae: 48093.4297
Epoch 4/100
336/336 - 1s - 4ms/step - loss: 46807.4688 - mae: 46807.4570 - val_loss: 45804.7344 - val_mae: 45804.7305
Epoch 5/100
336/336 - 1s - 4ms/step - loss: 44788.6797 - mae: 44788.6758 - val_loss: 43774.1836 - val_mae: 43774.1602
Epoch 6/100
336/336 - 1s - 3ms/step - loss: 43002.3594 - mae: 43002.3555 - val_loss: 41991.1484 - val_mae: 41991.1406
Epoch 7/100
336/336 - 1s - 3ms/step - loss: 41454.9180 - mae: 41454.9141 - val_loss: 40497.7969 - val_mae: 40497.7852
Epoch 8/100
336/336 - 1s - 4ms/step - loss: 40161.1797 - mae: 40161.1797 - val_loss: 39309.0586 - val_mae: 39309.0703
Epoch 9/100
336/336 - 1s - 4ms/step - loss: 39179.3398 -

In [78]:
predictions = model.predict(test_data.drop(columns=["id"]))
x=[]
for i in predictions:
    for j in i:
        x.append(j)

125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [79]:
sample=pd.DataFrame()

In [80]:
sample["id"]=test_data["id"]
sample['True_value']=x

In [81]:
sample

,id,True_value
0,Id_6252,84117.445312
1,Id_4684,142000.640625
2,Id_1731,49124.890625
3,Id_4742,21893.197266
4,Id_4521,160701.468750
...,...,...
3995,Id_4209,103489.484375
3996,Id_681,50367.851562
3997,Id_434,53984.097656
3998,Id_9309,45653.140625


In [82]:
sample.to_csv("submission.csv")